基于伟雁现在的feature在gbdt上的效果,加上了shift

前面主要是一些具体的思路过程，后面train_and_test是将前面的思路组合在一起

不同的数据集后续需要调参

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
OHE = OneHotEncoder(sparse=False)
pd.set_option('display.max_rows', 30)

In [2]:
train_path = 'data_feature/C_3.csv'
test_path = 'data_testing/C_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [3]:
train.head()

,Unnamed: 0,label,route,calculated_day,date,hr,weekday,115,102,109,104,112,111,103,122,precipitation
0,0,132.050000,C_3,236,7-19,8,1,6.703333,9.976667,10.743333,25.313333,30.343333,52.822500,2.505833,30.864286,0.0
1,1,219.150000,C_3,237,7-20,8,2,17.833333,12.845000,13.218333,32.235000,23.703333,36.149474,11.691053,63.107000,0.0
2,2,166.150000,C_3,238,7-21,8,3,9.853333,11.210000,11.466667,25.160000,15.066667,31.747143,12.790000,42.614762,0.0
3,3,161.786667,C_3,240,7-23,8,5,16.760000,11.832857,11.828571,26.501429,19.021429,29.680000,10.785909,43.795000,0.0
4,4,207.950000,C_3,241,7-24,8,6,10.098333,11.833333,12.073333,23.028333,17.815000,32.623000,8.675000,31.590909,0.0


In [4]:
# 月日如果用起来要one-hot，之后产生的feature就太多了而且感觉没必要，可能考虑is_special更合适
# 但是前后依赖这里怎么办呢？
# train[['month','day']] = train.date.str.split('-', expand=True)
def get_x_y(df):
    data = df.drop(columns=['Unnamed: 0', 'route', 'calculated_day', 'date'])
    ohe = OHE.fit_transform(data[['hr', 'weekday']])
    ohe_df = pd.DataFrame(ohe, columns= OHE.get_feature_names(input_features=['hr', 'weekday']))
    data = pd.concat([data, ohe_df],axis = 1)
    data = data.drop(columns=['hr', 'weekday'])
    x = data.drop(columns=['label'])
    y = data.label
    return x, y
    

In [5]:
train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

In [6]:
test_x

,115,102,109,104,112,111,103,122,precipitation,hr_8,hr_9,hr_17,hr_18,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,9.291111,36.812222,10.830000,41.146667,27.296250,21.788065,6.675484,29.930750,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,6.283333,8.340000,8.135000,19.785000,15.721667,20.287333,7.594333,32.362927,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,10.578000,18.864000,69.286000,67.678000,27.290000,27.313750,7.534583,29.848182,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,8.990000,14.816250,9.660000,24.726250,42.458333,22.890952,10.533333,30.849737,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,11.426667,13.003333,12.396667,22.243333,16.730000,26.895000,4.797857,37.987000,0.1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,9.171250,9.880000,8.305000,19.181250,20.370000,21.271667,10.950000,36.488947,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,10.770000,11.036429,11.332857,40.065385,26.874167,25.971304,2.571739,38.217838,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,9.291111,36.812222,10.830000,41.146667,27.296250,21.788065,6.675484,29.930750,1.8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,6.283333,8.340000,8.135000,19.785000,15.721667,20.287333,7.594333,32.362927,0.2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,10.578000,18.864000,69.286000,67.678000,27.290000,27.313750,7.534583,29.848182,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
train_x.isnull().sum().sort_values()

115              0
weekday_4        0
weekday_3        0
weekday_2        0
weekday_1        0
weekday_0        0
hr_18            0
hr_17            0
hr_9             0
hr_8             0
precipitation    0
122              0
103              0
111              0
112              0
104              0
109              0
102              0
weekday_5        0
weekday_6        0
dtype: int64

In [8]:
test_x.isnull().sum().sort_values()

115              0
weekday_4        0
weekday_3        0
weekday_2        0
weekday_1        0
weekday_0        0
hr_18            0
hr_17            0
hr_9             0
hr_8             0
precipitation    0
122              0
103              0
111              0
112              0
104              0
109              0
102              0
weekday_5        0
weekday_6        0
dtype: int64

In [9]:
def mape_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def scoring(reg, x, y):
    pred = reg.predict(x)
    return -mape_error(pred, y)

In [10]:
gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=6, random_state=0, loss='ls').fit(train_x, train_y)

In [11]:
# lr = 0.01,0.16887387929085965
# mape_error(test_y, gdbt.predict(test_x))

In [12]:
# lr = 0.1
mape_error(test_y, gdbt.predict(test_x))

0.16451955209454483

In [13]:
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
train_compare

,label,0
0,132.050000,145.665568
1,219.150000,200.844794
2,166.150000,173.410578
3,161.786667,185.700325
4,207.950000,208.269945
5,266.680000,249.642490
6,197.346667,190.222515
7,223.800000,207.195963
8,294.460000,287.554495
9,129.590000,164.591107


In [14]:
result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
test_compare

,label,0
0,194.430000,174.885517
1,188.860000,222.288700
2,168.170000,178.745836
3,169.465000,172.866827
4,196.430000,176.635468
5,NaN,214.490771
6,141.630000,177.912492
7,179.806667,180.235186
8,173.563333,199.703307
9,178.855000,169.641644


In [15]:
scaler = StandardScaler()
scaler.fit(train_x[['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']])
train_x[['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']] = scaler.transform(train_x[['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']])

In [16]:
train_x

,115,102,109,104,112,111,103,122,precipitation,hr_8,hr_9,hr_17,hr_18,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,-1.268120,-0.374101,-0.070886,-0.357790,0.210880,2.196034,-1.299569,-0.901596,-0.129679,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2.211417,1.189012,0.819079,0.298924,-0.426870,0.426760,1.433778,2.016773,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.283345,0.298010,0.189211,-0.372338,-1.256393,-0.040398,1.760804,0.161969,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.875864,0.637440,0.319346,-0.245066,-0.876551,-0.259755,1.164425,0.268795,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.206752,0.637699,0.407358,-0.574586,-0.992425,0.052545,0.536258,-0.835828,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,-0.210052,0.583809,0.540003,-0.255532,0.233078,2.471426,0.941540,-0.808689,-0.129679,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.891926,-1.365919,-1.021381,-0.730028,-1.278484,0.653055,-1.118193,-1.034698,-0.129679,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,-0.566794,0.323442,0.402563,-0.307188,-0.900380,-0.639405,1.051874,-0.672580,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,-1.001085,-1.024868,-0.683374,-0.631592,-0.592390,-0.497242,-1.050147,1.646622,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.122027,0.867488,0.873615,0.327862,-0.209164,-0.445458,1.949196,3.067134,-0.129679,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

In [18]:
mape_error(train_y, gdbt.predict(train_x))

0.10786627146418085

In [19]:
test_x[['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']] = scaler.transform(test_x[['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']])

In [20]:
# 可见data被scale之后会好很多
mape_error(test_y, gdbt.predict(test_x))

0.16136689463949525

In [21]:
result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
test_compare

,label,0
0,194.430000,190.127255
1,188.860000,198.920339
2,168.170000,189.462436
3,169.465000,175.393228
4,196.430000,169.789290
5,NaN,185.882848
6,141.630000,181.997898
7,179.806667,176.649185
8,173.563333,201.185904
9,178.855000,176.205615


In [22]:
# cv
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit

params = {'n_estimators': [10, 20, 30, 40, 50, 80, 100], 'max_depth': [2, 5, 8, 10, 15],
          'learning_rate': [0.1, 0.05, 0.01], 'loss': ['ls']}

regressor = GradientBoostingRegressor()
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
CLF = GridSearchCV(regressor, params, scoring=scoring, verbose=0, cv=cv)

In [23]:
CLF.fit(train_x, train_y)
print('best params:\n', CLF.best_params_)

mean_scores = np.array(CLF.cv_results_['mean_test_score'])
print('mean score', mean_scores)
print('best score', CLF.best_score_)
# 其实这里用cv出来的效果反而很差，应该是因为训练数据本来就少，还要拿一部分出来cv

best params:
 {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 8, 'n_estimators': 30}
mean score [-0.20458365 -0.20835486 -0.21015393 -0.21258636 -0.21419719 -0.21633031
 -0.22024207 -0.21186788 -0.22011365 -0.22606692 -0.22721199 -0.23074875
 -0.23530663 -0.23867096 -0.21897904 -0.23838512 -0.24168312 -0.24596553
 -0.24874379 -0.24848527 -0.2494926  -0.22501609 -0.24977484 -0.2614198
 -0.26498772 -0.26567581 -0.26579782 -0.26664105 -0.23316071 -0.26827913
 -0.27806749 -0.28301164 -0.28613633 -0.28593434 -0.28620579 -0.20402383
 -0.20455715 -0.20689252 -0.20863176 -0.20991068 -0.21287772 -0.21519368
 -0.20584871 -0.21038074 -0.21330687 -0.21705502 -0.21875446 -0.2232937
 -0.22523745 -0.20401605 -0.21959717 -0.23007754 -0.23823997 -0.24255236
 -0.24988572 -0.25144191 -0.20314614 -0.21970336 -0.23501645 -0.24471863
 -0.25123623 -0.25737791 -0.26119159 -0.20916561 -0.2332001  -0.25366466
 -0.26723441 -0.27764918 -0.2872701  -0.28683989 -0.20770813 -0.20645886
 -0.20538757 -0.20454256 -0

In [24]:
# {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 10, 'n_estimators': 40}
gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=6, random_state=0, loss='ls').fit(train_x, train_y)

In [25]:
mape_error(train_y, gdbt.predict(train_x))

0.08202491531432916

In [26]:
mape_error(test_y, gdbt.predict(test_x))

0.16779958597763753

In [51]:
# 先用固定的C的参数跑一下看效果
# A B 十字路口的数据比较多，可能是需要更复杂一点的hyper的
def train_and_test(train_path, test_path, scale_list, n_est=30, lr=0.1, depth=5, random=0, loss_func='ls'):
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
        
    train_x, train_y = get_x_y(train)
    test_x, test_y = get_x_y(test)
    
    
    scaler = StandardScaler()
    scaler.fit(train_x[scale_list])
    
    train_x[scale_list] = scaler.transform(train_x[scale_list])
    test_x[scale_list] = scaler.transform(test_x[scale_list])
    
#     print(train_x.isnull().sum().sort_values())
#     print(test_x.isnull().sum().sort_values())
    
    gdbt = GradientBoostingRegressor(n_estimators=n_est, learning_rate=lr, 
                                 max_depth=depth, random_state=random, loss=loss_func).fit(train_x, train_y)
    
    
    result = gdbt.predict(train_x)
    train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#     print(train_compare)
    print("train mape:", mape_error(train_y, gdbt.predict(train_x)))
    
    result = gdbt.predict(test_x)
    test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#     print(test_compare)
    print("test mape:", mape_error(test_y, gdbt.predict(test_x)))  

In [52]:
ori_path = 'data_feature/'
shift_path = 'train_shift/'
out_path = "train_augmentation/"
file_list = ['A_2.csv', 'A_3.csv', 'B_1.csv', 'B_3.csv', 'C_1.csv', 'C_3.csv']
left_prefix = '5min_shifted_'
right_prefix = '-5min_shifted_'

for file in file_list:
    left_df = pd.read_csv(shift_path + left_prefix + file)
    ori_df = pd.read_csv(ori_path + file)
    right_df = pd.read_csv(shift_path + right_prefix + file)

    aug_df = pd.concat([left_df, ori_df, right_df], axis = 0)
    aug_df = aug_df.drop(columns=['Unnamed: 0'])

    aug_df.to_csv(out_path + file)

In [53]:
train_path = 'data_feature/'
test_path = 'data_testing/'

scale_list = [['110', '123', '107', '108', '120', '117', 'precipitation'],
              ['110', '123', '107', '108', '119', '114', '118', '122', 'precipitation'],
              ['105', '100', '111', '103', '116', '101', '121', '106', '113', 'precipitation'],
              ['105', '100', '111', '103', '122', 'precipitation'],
              ['115', '102', '109', '104', '112', '111', '103', '116', '101', '121', '106', '113', 'precipitation'],
              ['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']]

for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i])

A_2.csv
train mape: 0.08391269024754676
test mape: 0.0967270551748314
A_3.csv
train mape: 0.08462432027534748
test mape: 0.21169600292898677
B_1.csv
train mape: 0.10428539027773352
test mape: 0.14568259200451397
B_3.csv
train mape: 0.08672401539461208
test mape: 0.14524607907523132
C_1.csv
train mape: 0.07498413746671939
test mape: 0.2069376080223797
C_3.csv
train mape: 0.10786627146418085
test mape: 0.16136689463949525


In [29]:
# A3, C_1的train和test效果差太多了，后续需要看看具体原因

# 后续各个model组要各自调参

# 0.15
# A_2.csv
# train mape: 0.06867742442319537
# test mape: 0.11368211626361499
# A_3.csv
# train mape: 0.06334452482742134
# test mape: 0.1979433833619259
# B_1.csv
# train mape: 0.08413829902862681
# test mape: 0.16377274857899568
# B_3.csv
# train mape: 0.06839501431970942
# test mape: 0.14131054062883014
# C_1.csv
# train mape: 0.0588710827921733
# test mape: 0.2031763715414268
# C_3.csv
# train mape: 0.07921782046971403
# test mape: 0.15206730528135995

# 0.1
# A_2.csv
# train mape: 0.08391269024754676
# test mape: 0.0967270551748314
# A_3.csv
# train mape: 0.08462432027534748
# test mape: 0.21169600292898677
# B_1.csv
# train mape: 0.10428539027773352
# test mape: 0.14568259200451397
# B_3.csv
# train mape: 0.08672401539461208
# test mape: 0.14524607907523132
# C_1.csv
# train mape: 0.07498413746671939
# test mape: 0.2069376080223797
# C_3.csv
# train mape: 0.10786627146418085
# test mape: 0.16136689463949525
    

In [54]:
train_path = 'train_augmentation/'
test_path = 'data_testing/'

scale_list = [['110', '123', '107', '108', '120', '117', 'precipitation'],
              ['110', '123', '107', '108', '119', '114', '118', '122', 'precipitation'],
              ['105', '100', '111', '103', '116', '101', '121', '106', '113', 'precipitation'],
              ['105', '100', '111', '103', '122', 'precipitation'],
              ['115', '102', '109', '104', '112', '111', '103', '116', '101', '121', '106', '113', 'precipitation'],
              ['115', '102', '109', '104', '112', '111', '103', '122', 'precipitation']]

# augmentation之后A3好了很多，C_1稍微好了一丢丢丢丢，A2和B1差了。可能仔细调参会好一点，比如接下来那几组参数，有些特别适合某几个数据集
for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i],n_est=30, lr=0.05, depth=5, random=0, loss_func='ls')

A_2.csv
train mape: 0.12814583571120874
test mape: 0.1106014273431828
A_3.csv
train mape: 0.13968907243924544
test mape: 0.16972102067501865
B_1.csv
train mape: 0.16113148652728204
test mape: 0.19237021382311234
B_3.csv
train mape: 0.13862696038399516
test mape: 0.12936121270558124
C_1.csv
train mape: 0.11320170558934314
test mape: 0.19671564305061248
C_3.csv
train mape: 0.1534350194453327
test mape: 0.16675393022838172


In [57]:
for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i],n_est=30, lr=0.1, depth=5, random=0, loss_func='ls')

A_2.csv
train mape: 0.10411983139858219
test mape: 0.11018740462243225
A_3.csv
train mape: 0.10434569788609369
test mape: 0.16459816534140162
B_1.csv
train mape: 0.12629447390378418
test mape: 0.16650244739430864
B_3.csv
train mape: 0.10718803065476103
test mape: 0.12094980886382471
C_1.csv
train mape: 0.08506739973494618
test mape: 0.20417613235016738
C_3.csv
train mape: 0.12425231855890614
test mape: 0.16726919931920678


In [60]:
for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i], n_est=35, lr=0.05, depth=5, random=0, loss_func='ls')

A_2.csv
train mape: 0.12258553698469894
test mape: 0.1073588633039206
A_3.csv
train mape: 0.131249343327142
test mape: 0.16754763052384392
B_1.csv
train mape: 0.15301441478462774
test mape: 0.19371716878470127
B_3.csv
train mape: 0.13085041967042002
test mape: 0.1227099098602407
C_1.csv
train mape: 0.10639212285420312
test mape: 0.19716992299700814
C_3.csv
train mape: 0.14861652507793208
test mape: 0.16603605058926083


In [65]:
for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i], n_est=35, lr=0.1, depth=5, random=0, loss_func='ls')

A_2.csv
train mape: 0.09944099390040682
test mape: 0.10859483492219799
A_3.csv
train mape: 0.09643750554423279
test mape: 0.16590514374352025
B_1.csv
train mape: 0.11767414540886
test mape: 0.1644979197859781
B_3.csv
train mape: 0.10165316389332738
test mape: 0.12243210691302608
C_1.csv
train mape: 0.07989550715290505
test mape: 0.2039345633668502
C_3.csv
train mape: 0.1152750678673875
test mape: 0.17003540520279561


In [66]:
for i in range(len(file_list)):
    file = file_list[i]
    print(file)
    train_and_test(train_path + file, test_path + file, scale_list[i], n_est=35, lr=0.1, depth=3, random=0, loss_func='ls')

A_2.csv
train mape: 0.13603289931433998
test mape: 0.1003126068680691
A_3.csv
train mape: 0.14062268725488175
test mape: 0.1757056761826131
B_1.csv
train mape: 0.18209721918954921
test mape: 0.13747785184919112
B_3.csv
train mape: 0.1543884812064363
test mape: 0.12555666805382237
C_1.csv
train mape: 0.12630090417884177
test mape: 0.20512397602712668
C_3.csv
train mape: 0.16432819051934716
test mape: 0.15183027210274566


In [ ]:
# 0.1，没有augmentation
# A_2.csv
# train mape: 0.08391269024754676
# test mape: 0.0967270551748314
# A_3.csv
# train mape: 0.08462432027534748
# test mape: 0.21169600292898677
# B_1.csv
# train mape: 0.10428539027773352
# test mape: 0.14568259200451397
# B_3.csv
# train mape: 0.08672401539461208
# test mape: 0.14524607907523132
# C_1.csv
# train mape: 0.07498413746671939
# test mape: 0.2069376080223797
# C_3.csv
# train mape: 0.10786627146418085
# test mape: 0.16136689463949525